# Prediccion de Precios de Casas usando CNN
Este notebook muestra un ejemplo de como entrenar una red neuronal convolucional (CNN) para predecir el precio de una casa utilizando tanto las caracteristicas numericas como las imagenes asociadas a cada propiedad.

## Carga de librerias y datos
Se utilizan librerias comunes de Python junto con **PyTorch** para definir el modelo y entrenarlo. Los datos se cargan desde los archivos CSV proporcionados y las imagenes se leen desde la carpeta `imgs`.

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

train_df = pd.read_csv('data/train_data.csv')
test_df = pd.read_csv('data/test_data.csv')


## Preparacion del conjunto de datos
Creamos una clase `Dataset` que combina las columnas numericas con las imagenes de cada casa. Cada propiedad tiene hasta cinco imagenes: bano, dormitorio, comedor, cocina y sala de estar. Las imagenes se redimensionan y se concatenan en un solo tensor.

In [ ]:
class HouseDataset(Dataset):
    def __init__(self, df, img_dir, is_train=True):
        self.df = df
        self.img_dir = img_dir
        self.is_train = is_train
        self.numeric_cols = [c for c in df.columns if c not in ['ID', 'price']]
        self.transform = transforms.Compose([
            transforms.Resize((64, 64)),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.df)

    def load_images(self, idx):
        property_id = int(self.df.iloc[idx]['ID'])
        parts = ['bath', 'bed', 'din', 'kitchen', 'living']
        images = []
        for p in parts:
            path = os.path.join(self.img_dir, f"{p}_{property_id}.jpg")
            img = Image.open(path).convert('RGB')
            images.append(self.transform(img))
        return torch.stack(images)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_tensor = self.load_images(idx)
        numeric = torch.tensor(row[self.numeric_cols].fillna(0).to_numpy(), dtype=torch.float32)
        if self.is_train:
            target = torch.tensor(row['price'], dtype=torch.float32)
            return img_tensor, numeric, target
        else:
            return img_tensor, numeric


## Definicion del modelo
El modelo consta de una red convolucional que procesa cada imagen de manera independiente seguida de una capa totalmente conectada que incorpora las caracteristicas numericas.

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
        )
        cnn_output = 32 * 16 * 16  # 64x64 -> 32x32 -> 16x16
        self.fc = nn.Sequential(
            nn.Linear(5 * cnn_output + num_features, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, images, numeric):
        batch_size, num_imgs, c, h, w = images.shape
        images = images.view(batch_size * num_imgs, c, h, w)
        features = self.cnn(images)
        features = features.view(batch_size, num_imgs * features.shape[1])
        x = torch.cat([features, numeric], dim=1)
        return self.fc(x).squeeze()


## Entrenamiento
Se preparan los *DataLoader* para entrenamiento y validacion. Luego se entrena la red durante varias epocas utilizando el optimizador Adam y la funcion de perdida MSE.

In [ ]:
train_dataset = HouseDataset(train_df, 'imgs', is_train=True)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

model = CNNModel(num_features=len(train_dataset.numeric_cols))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(5):
    for images, numeric, target in train_loader:
        pred = model(images, numeric)
        loss = criterion(pred, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')


## Generacion de predicciones
Se cargan los datos de prueba y se guarda el archivo `submission.csv` con el formato solicitado.

In [ ]:
test_dataset = HouseDataset(test_df, 'imgs', is_train=False)
test_loader = DataLoader(test_dataset, batch_size=8)

model.eval()
preds = []
with torch.no_grad():
    for images, numeric in test_loader:
        pred = model(images, numeric)
        preds.extend(pred.cpu().numpy())

submission = pd.DataFrame({'ID': test_df['ID'], 'price': preds})
submission.to_csv('submission.csv', index=False)
